# Data Manipulation
In questa sezione modifico il dataframe iniziale aggiungendo latitudine e longitudine in base agli address. 
Utilizzo geopandas.
La seconda sezione mi permette di modificare il dataset e creare un geojson per poterlo visualizzare in mapbox.

In [1]:
import geopandas
from geopy.geocoders import Nominatim
import pandas as pd
import warnings
import json
import requests
import time
import geopandas
import warnings
warnings.filterwarnings('ignore')

## Modifico le stringhe di address

In [115]:
df = pd.read_csv("./trento/2020.csv")

In [116]:
#df_complete = df_complete.drop(df_complete.columns[[0]],1)
df
#df_complete.to_csv("./trento/2020.csv", index=False)

,title,subtitle,link,timestamp,text,keywords,address
0,Arrivano gli alberi di Natale: chiusa la strad...,La strada sarà chiusa durante le operazioni di...,https://www.trentotoday.it/attualita/strada-bo...,08 novembre 2020 10:15,Strada chiusa per trasporto alberi di Natale. ...,"['strade', 'trasporti', 'viabilità']","Trento,Piazza del Duomo"
1,Tamponi: anche a Trento arriva l'esercito,Operazione del Ministero della Difesa in suppo...,https://www.trentotoday.it/attualita/esercito-...,08 novembre 2020 09:56,Arriva anche a Trento il punto tamponi gestito...,"['alpini', 'esercito', 'tamponi']","Oltrefersina,Via Fersina"
2,Bollettino coronavirus: 251 nuovi casi in Tren...,Non accenna a diminuire il contagio: ecco i da...,https://www.trentotoday.it/cronaca/bollettino-...,07 novembre 2020 16:43,"Non accenna ad ""appiattirsi"" la linea dei cont...","['bollettino', 'contagi', 'coronavirus']","Oltrefersina,Largo Medaglie d'oro"
3,"Precipita in ferrata: morta Federica Marcolla,...","Tragico incidente sulla ""Che Guevara"", probabi...",https://www.trentotoday.it/cronaca/federica-ma...,07 novembre 2020 15:50,"E' Federica Marcolla, medico e mamma di due ba...","['incidenti montagna', 'morti', 'morti in mont...","Dro,Via Monte Casale"
4,"Dro, precipita dalla ferrata per 200 metri e m...","Drammatico incidente alla ferrata Che Guevara,...",https://www.trentotoday.it/cronaca/ragazza-mor...,07 novembre 2020 14:39,Mattinata drammatica sul Monte Casale: una rag...,"['ferrate', 'incidenti montagna', 'morti']","Dro,Via Monte Casale"
...,...,...,...,...,...,...,...
2980,"Ticket sanitario, nuove regole e esenzioni per...",Con il 2020 vengono abrogati i ticket per rice...,https://www.trentotoday.it/attualita/esenzione...,02 gennaio 2020 09:27,Niente più ticket per ricette e prestazioni sp...,"['salute', 'sanità', 'ticket']",NaN
2981,Saldi invernali 2020 a Trento: al via i primi ...,Parte la caccia ai super sconti in Trentino. E...,https://www.trentotoday.it/attualita/saldi-inv...,02 gennaio 2020 07:58,Partono i saldi invernali a Trento e in tutto ...,"['promozioni', 'saldi']",NaN
2982,Tuffo del Primo dell'Anno a Riva del Garda: 12...,"In 125 si sono ritrovati in piazza Catena, per...",https://www.trentotoday.it/attualita/tuffo-cap...,01 gennaio 2020 17:33,"Per tanti è un ""toccasana"", per alcuni una fol...","['riva del garda', 'tuffo capodanno', 'tuffo m...","Trento,Piazza Catena"
2983,Capodanno: un successo le feste in piazza a Tr...,"Rock in Movie a Trento, dj set e luci dell'ass...",https://www.trentotoday.it/attualita/festa-cap...,01 gennaio 2020 14:25,"Musica e cinema, ma soprattutto tanta voglia d...",['capodanno'],"Trento,Piazza del Duomo"


# Implemento geocoder con openstreetmap
Il geocoder di osm si applica collegandosi ad un api online. Per implementarlo creeró una classe che mi permetterá di fare una richiesta e di elaborare il risultato.
Questo tipo di geocoder é identico a geopy.geocoder.nominatim che sfrutta openmap. Semplicemente é un esercizio di stile capire come funzioni un api.

Per info andare su https://nominatim.openstreetmap.org/ui/search.html nella sezione API
YT video https://www.youtube.com/watch?v=R3jkosrWsE8
L'ho implementato in geocoder.py

In [124]:

# Per rispetto della policy bisogna impostare 1 secondo di delay tra una request e ĺ'altra

# Geocoder class
class Geocoder:
    #base url
    base_url = 'https://nominatim.openstreetmap.org/search'
    
    def fetch(self, address):
        #string query params
        params ={
            'q': address,
            'format': 'geocodejson'
        }
        res = requests.get(url = self.base_url, params = params)
        print('Status code: {}'.format(res.status_code))
        
        if(res.status_code == 200):
            return res
        else:
            return ''
    
    def parse(self, res):
        try:
            
            label = json.dumps(res['features'][0]['properties']['geocoding']['label'], indent = 2)
            coordinates = json.dumps(res['features'][0]['geometry']['coordinates'], indent = 2).replace('\n', '').replace('[', '').replace(']', '').strip().split(',')
            obj = {
                'label': label,
                'coordinates': coordinates
            }
            return (label, coordinates);
        except:
            return ('', ['','']);
        
        
    
    def run(self, address):
        print(address)
        res = self.fetch(address).json()
        time.sleep(1)
        return self.parse(res)
        # Per rispetto delle norme policy
        

In [129]:
geocoder = Geocoder()
df = pd.read_csv("./trento/2020.csv")
label = []
coordinates = []

#label, geo  = geocoder.run('oltrefersina, largo medaglie d')
#label, coordinates = df['address'].apply(lambda x: geocoder.run(x))
for address in  df['address']:
    lab, coor = geocoder.run(str(address))
    label.append(lab)
    coordinates.append(coor)
df['location'] = label
df['coordinates'] = coordinates

df.to_csv('2020parsed.csv', index = False)

Trento,Piazza del Duomo
Status code: 200
Oltrefersina,Via Fersina
Status code: 200
Oltrefersina,Largo Medaglie d'oro
Status code: 200
Dro,Via Monte Casale
Status code: 200
Dro,Via Monte Casale
Status code: 200
Trento,Via Giannantonio Manci
Status code: 200
Trento,Via Giuseppe Verdi
Status code: 200
Trento,Via Oss Mazzurana
Status code: 200
Trento,Piazza del Duomo
Status code: 200
nan
Status code: 200
nan
Status code: 200
nan
Status code: 200
San Giuseppe,Piazza di Fiera
Status code: 200
Trento,Piazza del Duomo
Status code: 200
nan
Status code: 200
nan
Status code: 200
nan
Status code: 200
nan
Status code: 200
Oltrefersina,Largo Medaglie d'oro
Status code: 200
Trento,Piazza Dante
Status code: 200
Pergine Valsugana
Status code: 200
Trento,Piazza Dante
Status code: 200
nan
Status code: 200
Riva del Garda
Status code: 200
nan
Status code: 200
Trento,Piazza Dante
Status code: 200
nan
Status code: 200
nan
Status code: 200
Oltrefersina,Viale Verona
Status code: 200
Trento,Tangenziale Ovest
St

# Unificazione dataset
Uniformo i dataset dell'anno al dataset generale in modo da poterlo unire.

In [2]:
# Importo il dataset
df2020 = pd.read_csv('trento/2020parsed.csv')
df2020

,title,subtitle,link,timestamp,text,keywords,address,location,coordinates
0,Arrivano gli alberi di Natale: chiusa la strad...,La strada sarà chiusa durante le operazioni di...,https://www.trentotoday.it/attualita/strada-bo...,08 novembre 2020 10:15,Strada chiusa per trasporto alberi di Natale. ...,"['strade', 'trasporti', 'viabilità']","Trento,Piazza del Duomo","""Piazza del Duomo, Centro storico Trento, Tren...","['11.1210924', ' 46.0676474']"
1,Tamponi: anche a Trento arriva l'esercito,Operazione del Ministero della Difesa in suppo...,https://www.trentotoday.it/attualita/esercito-...,08 novembre 2020 09:56,Arriva anche a Trento il punto tamponi gestito...,"['alpini', 'esercito', 'tamponi']","Oltrefersina,Via Fersina",NaN,"['', '']"
2,Bollettino coronavirus: 251 nuovi casi in Tren...,Non accenna a diminuire il contagio: ecco i da...,https://www.trentotoday.it/cronaca/bollettino-...,07 novembre 2020 16:43,"Non accenna ad ""appiattirsi"" la linea dei cont...","['bollettino', 'contagi', 'coronavirus']","Oltrefersina,Largo Medaglie d'oro",NaN,"['', '']"
3,"Precipita in ferrata: morta Federica Marcolla,...","Tragico incidente sulla ""Che Guevara"", probabi...",https://www.trentotoday.it/cronaca/federica-ma...,07 novembre 2020 15:50,"E' Federica Marcolla, medico e mamma di due ba...","['incidenti montagna', 'morti', 'morti in mont...","Dro,Via Monte Casale","""Monte Casale, Via Ferrata Ernesto Che Guevara...","['10.9276592', ' 46.031499']"
4,"Dro, precipita dalla ferrata per 200 metri e m...","Drammatico incidente alla ferrata Che Guevara,...",https://www.trentotoday.it/cronaca/ragazza-mor...,07 novembre 2020 14:39,Mattinata drammatica sul Monte Casale: una rag...,"['ferrate', 'incidenti montagna', 'morti']","Dro,Via Monte Casale","""Monte Casale, Via Ferrata Ernesto Che Guevara...","['10.9276592', ' 46.031499']"
...,...,...,...,...,...,...,...,...,...
2980,"Ticket sanitario, nuove regole e esenzioni per...",Con il 2020 vengono abrogati i ticket per rice...,https://www.trentotoday.it/attualita/esenzione...,02 gennaio 2020 09:27,Niente più ticket per ricette e prestazioni sp...,"['salute', 'sanità', 'ticket']",NaN,"""Nanno, Comunit\u00e0 della Val di Non, Provin...","['11.0480288', ' 46.3144754']"
2981,Saldi invernali 2020 a Trento: al via i primi ...,Parte la caccia ai super sconti in Trentino. E...,https://www.trentotoday.it/attualita/saldi-inv...,02 gennaio 2020 07:58,Partono i saldi invernali a Trento e in tutto ...,"['promozioni', 'saldi']",NaN,"""Nanno, Comunit\u00e0 della Val di Non, Provin...","['11.0480288', ' 46.3144754']"
2982,Tuffo del Primo dell'Anno a Riva del Garda: 12...,"In 125 si sono ritrovati in piazza Catena, per...",https://www.trentotoday.it/attualita/tuffo-cap...,01 gennaio 2020 17:33,"Per tanti è un ""toccasana"", per alcuni una fol...","['riva del garda', 'tuffo capodanno', 'tuffo m...","Trento,Piazza Catena","""Piazza Catena, Piazza Tre Novembre, Alboletta...","['10.838915858870708', ' 45.88417045']"
2983,Capodanno: un successo le feste in piazza a Tr...,"Rock in Movie a Trento, dj set e luci dell'ass...",https://www.trentotoday.it/attualita/festa-cap...,01 gennaio 2020 14:25,"Musica e cinema, ma soprattutto tanta voglia d...",['capodanno'],"Trento,Piazza del Duomo","""Piazza del Duomo, Centro storico Trento, Tren...","['11.1210924', ' 46.0676474']"


In [3]:
df = pd.read_csv("./trento/dfTextProcessed3.csv")
df

,title,subtitle,link,timestamp,text,keywords,address,location,point,latitude,longitude,year,month,day,list_keywords
0,Recuperato il corpo di Daniele Catorci,"Il giovane alpinista, dottorando all'Universit...",http://www.trentotoday.it/cronaca/daniele-cato...,2020/06/29,stato recuperato pomeriggio lunedì 29 giugno ...,"['incidenti montagna', 'morti montagna']","Gardolo,via Giambattista Unterveger",NaN,"(0.0, 0.0, 0.0)",0.000000,0.000000,2020,6,29,"['Incidenti', 'Montagna', 'Morti', 'Montagna']"
1,Cade nel fiume: ripescato vivo un chilometro e...,Diversi corpi di Vigili del Fuoco hanno presid...,http://www.trentotoday.it/cronaca/cade-fiume-b...,2020/06/29,stato ripescato vivo acque dellisarco cinquan...,"['annegamenti', 'fiumi']","Gardolo,Via Alto Adige","Via Alto Adige, Spini di Gardolo, Trento, Terr...","(46.1148209, 11.1064463, 0.0)",46.114821,11.106446,2020,6,29,"['Annegamenti', 'Fiumi']"
2,"La Provincia: ""L'orso del video sarà catturato...",La nota della Pat sul video che sta spopolando...,http://www.trentotoday.it/green/orso-video-cat...,2020/06/29,orso segue pacificamente unescursionista bosc...,"['orsi', 'orso', 'video']",NaN,NaN,"(0.0, 0.0, 0.0)",0.000000,0.000000,2020,6,29,"['Orsi', 'Orso', 'Video']"
3,Sagra della braciola d'orso con il patrocinio ...,"La notizia ha gettato benzina sul fuoco, ma l'...",http://www.trentotoday.it/attualita/sagra-brac...,2020/06/29,"sagra braciola orso, patrocinio provincia aut...","['fake news', 'orsi']","Trento,Piazza Dante","Trento, Piazza Dante, Centro storico Trento, T...","(46.0722416, 11.1193186, 0.0)",46.072242,11.119319,2020,6,29,"['Fake', 'News', 'Orsi']"
4,Dramma sul Gran Paradiso: individuato lo zaino...,Nel crepaccio dove il giovane alpinista è cadu...,http://www.trentotoday.it/cronaca/morto-gran-p...,2020/06/29,operazioni 400 metri quota gran paradiso recu...,"['Daniele Catorci', 'Argomenti:', 'morti', 'mo...","Oltrefersina,Via Aosta",NaN,"(0.0, 0.0, 0.0)",0.000000,0.000000,2020,6,29,"['Daniele', 'Catorci', 'Argomenti', 'Morti', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27360,Progettone: niente accordo sugli stipendi dei ...,I sindacati non hanno raggiunto l'intesa con l...,http://www.trentotoday.it/economia/progettone-...,2011/11/09,nan,['lavoro'],NaN,NaN,"(0.0, 0.0, 0.0)",0.000000,0.000000,2011,11,9,['Lavoro']
27361,Le nuove frontiere di internet: due giorni di ...,Presenti a Trento oltre cento relatori per dis...,http://www.trentotoday.it/cronaca/internet-con...,2011/11/09,nan,['tecnologie'],NaN,NaN,"(0.0, 0.0, 0.0)",0.000000,0.000000,2011,11,9,['Tecnologie']
27362,Centro islamico a Gardolo: il Consiglio di Sta...,La comunità musulmana trentina potrà utilizzar...,http://www.trentotoday.it/cronaca/moschea-gard...,2011/11/09,gardolo centro islamico consiglio stato infat...,['religione'],NaN,NaN,"(0.0, 0.0, 0.0)",0.000000,0.000000,2011,11,9,['Religione']
27363,Piedibus: il servizio gratuito arriva anche a ...,"Hanno già aderito trentasei bambini, suddivisi...",http://www.trentotoday.it/cronaca/piedibus-mea...,2011/11/09,grazie disponibilità sedici genitori volontar...,['trasporti'],NaN,NaN,"(0.0, 0.0, 0.0)",0.000000,0.000000,2011,11,9,['Trasporti']


In [7]:
## cambio timestamp 2020
# Converto le date da timestamp in yyyy/MM/dd
# Sacrifico l'orario poiché é l'ora della pubblicazione del articolo
import datetime
import re

def funzioneConvertiData(stringa):
    month = {	'gennaio':'01',
                'febbraio':'02',
                'marzo':'03',
                'aprile':'04',
                'maggio':'05',
                'giugno':'06',
                'luglio':'07',
                'agosto':'08',
                'settembre':'09',
                'ottobre':'10',
                'novembre':'11',
                'dicembre':'12'
            }
    for ita, eng in month.items():
        stringa = stringa.replace(ita.lower(), eng)
    return stringa

def conversioneInData(stringa): 
    #d = datetime.datetime.strptime(stringa, '%d %m %Y %H:%M')
    d = datetime.datetime.strptime(stringa, '%Y/%m/%d')
    year = d.strftime('%Y')
    month = d.strftime('%m')
    day = d.strftime('%d')
    return d.strftime('%Y/%m/%d')
    
df2020['timestamp'] =  df2020['timestamp'].apply(lambda x: funzioneConvertiData(x))
df2020['timestamp'] =  df2020['timestamp'].apply(lambda x: conversioneInData(x))
df2020['year'] = df2020['timestamp'].apply(lambda x: datetime.datetime.strptime(x, '%Y/%m/%d').strftime('%Y'))
df2020['month'] = df2020['timestamp'].apply(lambda x: datetime.datetime.strptime(x, '%Y/%m/%d').strftime('%m'))
df2020['day'] = df2020['timestamp'].apply(lambda x: datetime.datetime.strptime(x, '%Y/%m/%d').strftime('%d'))

# converto keyword in list_keyword
df2020["list_keywords"] = df2020["keywords"].apply(lambda x: re.findall('\'([^\']*)\'', x)) # Transform string into list



,title,subtitle,link,timestamp,text,keywords,address,location,coordinates,year,month,day,list_keywords
0,Arrivano gli alberi di Natale: chiusa la strad...,La strada sarà chiusa durante le operazioni di...,https://www.trentotoday.it/attualita/strada-bo...,2020/11/08,Strada chiusa per trasporto alberi di Natale. ...,"['strade', 'trasporti', 'viabilità']","Trento,Piazza del Duomo","""Piazza del Duomo, Centro storico Trento, Tren...","['11.1210924', ' 46.0676474']",2020,11,08,"[strade, trasporti, viabilità]"
1,Tamponi: anche a Trento arriva l'esercito,Operazione del Ministero della Difesa in suppo...,https://www.trentotoday.it/attualita/esercito-...,2020/11/08,Arriva anche a Trento il punto tamponi gestito...,"['alpini', 'esercito', 'tamponi']","Oltrefersina,Via Fersina",NaN,"['', '']",2020,11,08,"[alpini, esercito, tamponi]"
2,Bollettino coronavirus: 251 nuovi casi in Tren...,Non accenna a diminuire il contagio: ecco i da...,https://www.trentotoday.it/cronaca/bollettino-...,2020/11/07,"Non accenna ad ""appiattirsi"" la linea dei cont...","['bollettino', 'contagi', 'coronavirus']","Oltrefersina,Largo Medaglie d'oro",NaN,"['', '']",2020,11,07,"[bollettino, contagi, coronavirus]"
3,"Precipita in ferrata: morta Federica Marcolla,...","Tragico incidente sulla ""Che Guevara"", probabi...",https://www.trentotoday.it/cronaca/federica-ma...,2020/11/07,"E' Federica Marcolla, medico e mamma di due ba...","['incidenti montagna', 'morti', 'morti in mont...","Dro,Via Monte Casale","""Monte Casale, Via Ferrata Ernesto Che Guevara...","['10.9276592', ' 46.031499']",2020,11,07,"[incidenti montagna, morti, morti in montagna]"
4,"Dro, precipita dalla ferrata per 200 metri e m...","Drammatico incidente alla ferrata Che Guevara,...",https://www.trentotoday.it/cronaca/ragazza-mor...,2020/11/07,Mattinata drammatica sul Monte Casale: una rag...,"['ferrate', 'incidenti montagna', 'morti']","Dro,Via Monte Casale","""Monte Casale, Via Ferrata Ernesto Che Guevara...","['10.9276592', ' 46.031499']",2020,11,07,"[ferrate, incidenti montagna, morti]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2980,"Ticket sanitario, nuove regole e esenzioni per...",Con il 2020 vengono abrogati i ticket per rice...,https://www.trentotoday.it/attualita/esenzione...,2020/01/02,Niente più ticket per ricette e prestazioni sp...,"['salute', 'sanità', 'ticket']",NaN,"""Nanno, Comunit\u00e0 della Val di Non, Provin...","['11.0480288', ' 46.3144754']",2020,01,02,"[salute, sanità, ticket]"
2981,Saldi invernali 2020 a Trento: al via i primi ...,Parte la caccia ai super sconti in Trentino. E...,https://www.trentotoday.it/attualita/saldi-inv...,2020/01/02,Partono i saldi invernali a Trento e in tutto ...,"['promozioni', 'saldi']",NaN,"""Nanno, Comunit\u00e0 della Val di Non, Provin...","['11.0480288', ' 46.3144754']",2020,01,02,"[promozioni, saldi]"
2982,Tuffo del Primo dell'Anno a Riva del Garda: 12...,"In 125 si sono ritrovati in piazza Catena, per...",https://www.trentotoday.it/attualita/tuffo-cap...,2020/01/01,"Per tanti è un ""toccasana"", per alcuni una fol...","['riva del garda', 'tuffo capodanno', 'tuffo m...","Trento,Piazza Catena","""Piazza Catena, Piazza Tre Novembre, Alboletta...","['10.838915858870708', ' 45.88417045']",2020,01,01,"[riva del garda, tuffo capodanno, tuffo merla]"
2983,Capodanno: un successo le feste in piazza a Tr...,"Rock in Movie a Trento, dj set e luci dell'ass...",https://www.trentotoday.it/attualita/festa-cap...,2020/01/01,"Musica e cinema, ma soprattutto tanta voglia d...",['capodanno'],"Trento,Piazza del Duomo","""Piazza del Duomo, Centro storico Trento, Tren...","['11.1210924', ' 46.0676474']",2020,01,01,[capodanno]


In [10]:
# creo latitude e longitude
df2020['coordinates'] = df2020['coordinates'].apply(lambda x: re.findall('\'([^\']*)\'', x))
df2020['longitude'] = df2020['coordinates'].apply(lambda x: x[0])
df2020['latitude'] = df2020['coordinates'].apply(lambda x: x[1])
df2020['point'] = df2020['coordinates']


TypeError: expected string or bytes-like object

In [11]:
df2020=df2020.drop('coordinates', axis=1)

df2020

,title,subtitle,link,timestamp,text,keywords,address,location,year,month,day,list_keywords,longitude,latitude,point
0,Arrivano gli alberi di Natale: chiusa la strad...,La strada sarà chiusa durante le operazioni di...,https://www.trentotoday.it/attualita/strada-bo...,2020/11/08,Strada chiusa per trasporto alberi di Natale. ...,"['strade', 'trasporti', 'viabilità']","Trento,Piazza del Duomo","""Piazza del Duomo, Centro storico Trento, Tren...",2020,11,08,"[strade, trasporti, viabilità]",11.1210924,46.0676474,"[11.1210924, 46.0676474]"
1,Tamponi: anche a Trento arriva l'esercito,Operazione del Ministero della Difesa in suppo...,https://www.trentotoday.it/attualita/esercito-...,2020/11/08,Arriva anche a Trento il punto tamponi gestito...,"['alpini', 'esercito', 'tamponi']","Oltrefersina,Via Fersina",NaN,2020,11,08,"[alpini, esercito, tamponi]",,,"[, ]"
2,Bollettino coronavirus: 251 nuovi casi in Tren...,Non accenna a diminuire il contagio: ecco i da...,https://www.trentotoday.it/cronaca/bollettino-...,2020/11/07,"Non accenna ad ""appiattirsi"" la linea dei cont...","['bollettino', 'contagi', 'coronavirus']","Oltrefersina,Largo Medaglie d'oro",NaN,2020,11,07,"[bollettino, contagi, coronavirus]",,,"[, ]"
3,"Precipita in ferrata: morta Federica Marcolla,...","Tragico incidente sulla ""Che Guevara"", probabi...",https://www.trentotoday.it/cronaca/federica-ma...,2020/11/07,"E' Federica Marcolla, medico e mamma di due ba...","['incidenti montagna', 'morti', 'morti in mont...","Dro,Via Monte Casale","""Monte Casale, Via Ferrata Ernesto Che Guevara...",2020,11,07,"[incidenti montagna, morti, morti in montagna]",10.9276592,46.031499,"[10.9276592, 46.031499]"
4,"Dro, precipita dalla ferrata per 200 metri e m...","Drammatico incidente alla ferrata Che Guevara,...",https://www.trentotoday.it/cronaca/ragazza-mor...,2020/11/07,Mattinata drammatica sul Monte Casale: una rag...,"['ferrate', 'incidenti montagna', 'morti']","Dro,Via Monte Casale","""Monte Casale, Via Ferrata Ernesto Che Guevara...",2020,11,07,"[ferrate, incidenti montagna, morti]",10.9276592,46.031499,"[10.9276592, 46.031499]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2980,"Ticket sanitario, nuove regole e esenzioni per...",Con il 2020 vengono abrogati i ticket per rice...,https://www.trentotoday.it/attualita/esenzione...,2020/01/02,Niente più ticket per ricette e prestazioni sp...,"['salute', 'sanità', 'ticket']",NaN,"""Nanno, Comunit\u00e0 della Val di Non, Provin...",2020,01,02,"[salute, sanità, ticket]",11.0480288,46.3144754,"[11.0480288, 46.3144754]"
2981,Saldi invernali 2020 a Trento: al via i primi ...,Parte la caccia ai super sconti in Trentino. E...,https://www.trentotoday.it/attualita/saldi-inv...,2020/01/02,Partono i saldi invernali a Trento e in tutto ...,"['promozioni', 'saldi']",NaN,"""Nanno, Comunit\u00e0 della Val di Non, Provin...",2020,01,02,"[promozioni, saldi]",11.0480288,46.3144754,"[11.0480288, 46.3144754]"
2982,Tuffo del Primo dell'Anno a Riva del Garda: 12...,"In 125 si sono ritrovati in piazza Catena, per...",https://www.trentotoday.it/attualita/tuffo-cap...,2020/01/01,"Per tanti è un ""toccasana"", per alcuni una fol...","['riva del garda', 'tuffo capodanno', 'tuffo m...","Trento,Piazza Catena","""Piazza Catena, Piazza Tre Novembre, Alboletta...",2020,01,01,"[riva del garda, tuffo capodanno, tuffo merla]",10.838915858870708,45.88417045,"[10.838915858870708, 45.88417045]"
2983,Capodanno: un successo le feste in piazza a Tr...,"Rock in Movie a Trento, dj set e luci dell'ass...",https://www.trentotoday.it/attualita/festa-cap...,2020/01/01,"Musica e cinema, ma soprattutto tanta voglia d...",['capodanno'],"Trento,Piazza del Duomo","""Piazza del Duomo, Centro storico Trento, Tren...",2020,01,01,[capodanno],11.1210924,46.0676474,"[11.1210924, 46.0676474]"


In [21]:
df = pd.read_csv("./trento/dfTextProcessed3.csv")
#df.drop((df['year']=='2020').index)
df = df.drop(df[df['year']==2020].index)

In [25]:
df = pd.concat([df2020, df])

In [26]:
df.to_csv('total.csv', index=False)

# Pulizia dadaset

In [2]:
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import numpy as np

In [3]:
df = pd.read_csv("./trento/dfTextProcessed3.csv")

In [6]:
# Droppo le righe che non sono indicizzate a nessuna posizione
df.dropna(subset = ["location"], inplace=True)

In [7]:
#Elimino i null
df = df.fillna("")

Dall'analisi fatta su www.geojson.io molte feature non sono indicizzate al solo trentino. Quindi vado a studiare quelle che superano determinate coordinate geografiche

In [10]:
df = df[(df['latitude']<=47) & (df['latitude']>=45)]

In [11]:
df = df[(df['longitude']<=12) & (df['longitude']>=10)]

Quello che mi serve fare ora é una nuova colonna "classes" che contenga una stringa con le keywords piú l'anno separati da spazio

In [6]:
import re
sentences = df["keywords"].apply(lambda x: re.findall('\'([^\']*)\'', x)) # Transform string into list
years = df["year"].apply(lambda x: 'y{}'.format(str(x)))

In [13]:
years

1        y2020
3        y2020
6        y2020
7        y2020
9        y2020
         ...  
27301    y2011
27305    y2011
27307    y2011
27308    y2011
27310    y2011
Name: year, Length: 15544, dtype: object

In [7]:
sentencesString = sentences.apply(lambda x: ' '.join(x) + ' circle ')
classes = sentencesString + years

df["classes"] = classes

## Creazione del geojson

In [16]:
df

,title,subtitle,link,timestamp,text,keywords,address,location,point,latitude,longitude,year,month,day,list_keywords,classes
1,Cade nel fiume: ripescato vivo un chilometro e...,Diversi corpi di Vigili del Fuoco hanno presid...,http://www.trentotoday.it/cronaca/cade-fiume-b...,2020/06/29,stato ripescato vivo acque dellisarco cinquan...,"['annegamenti', 'fiumi']","Gardolo,Via Alto Adige","Via Alto Adige, Spini di Gardolo, Trento, Terr...","(46.1148209, 11.1064463, 0.0)",46.114821,11.106446,2020,6,29,"['annegamenti', 'fiumi']",annegamenti fiumi circle y2020
3,Sagra della braciola d'orso con il patrocinio ...,"La notizia ha gettato benzina sul fuoco, ma l'...",http://www.trentotoday.it/attualita/sagra-brac...,2020/06/29,"sagra braciola orso, patrocinio provincia aut...","['fake news', 'orsi']","Trento,Piazza Dante","Trento, Piazza Dante, Centro storico Trento, T...","(46.0722416, 11.1193186, 0.0)",46.072242,11.119319,2020,6,29,"['fake', 'news', 'orsi']",fake news orsi circle y2020
6,L'orso si avvicina all'escursionista e lo segu...,Il video che ha fatto il giro del web è stato ...,http://www.trentotoday.it/cronaca/orso-segue-e...,2020/06/29,escursionista inseguito dallorso video fatto ...,"['orsi', 'orso', 'video']",Cavedago,"Cavedago, Comunità della Paganella, Provincia ...","(46.1852103, 11.0314805, 0.0)",46.185210,11.031481,2020,6,29,"['orsi', 'orso', 'video']",orsi orso video circle y2020
7,Bonus baby sitter anche ai nonni,A poco più di un mese dalla scadenza del bonus...,http://www.trentotoday.it/attualita/bonus-nonn...,2020/06/29,"bonus baby sitter nonni, zii parenti vari pen...","['coronavirus', 'famiglia']","Trento,Piazza del Duomo","Piazza del Duomo, Centro storico Trento, Trent...","(46.0673656, 11.1216994, 0.0)",46.067366,11.121699,2020,6,29,"['coronavirus', 'famiglia']",coronavirus famiglia circle y2020
9,Riva del Garda: 1.200 bottiglie di spumante ca...,"Temperatura costante, niente luce ed il dolce ...",http://www.trentotoday.it/attualita/spumante-f...,2020/06/29,216 bottiglie spumante fondo lago garda tratt...,"['notizie curiose', 'spumante', 'vini']","Riva del Garda,SS240, 17","La Paz, Viale Giosuè Carducci, Alboletta, Riva...","(45.88552155, 10.847790698161678, 0.0)",45.885522,10.847791,2020,6,29,"['notizie', 'curiose', 'spumante', 'vini']",notizie curiose spumante vini circle y2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27301,Trento città del Natale: tutte le iniziative f...,Musica e giochi per bambini in piazza Duomo. A...,http://www.trentotoday.it/cronaca/natale-capod...,2011/11/17,quest’anno trento trasforma città natale,['manifestazioni'],Trento,"Trento, Territorio Val d'Adige, Provincia di T...","(46.0664228, 11.1257601, 0.0)",46.066423,11.125760,2011,11,17,['manifestazioni'],manifestazioni circle y2011
27305,Attentato ai ripetitori sul Calisio: indagano ...,L'incendio innescato con la benzina e dei time...,http://www.trentotoday.it/cronaca/incendio-att...,2011/11/17,attentato incendiario stato compiuto scorsa n...,['incendi'],Argentario,"Argentario, Strada Provinciale 225, Albiano, C...","(46.1249257, 11.1797764, 0.0)",46.124926,11.179776,2011,11,17,['incendi'],incendi circle y2011
27307,Uomo muore schiacciato dal suo trattore,La vittima è un settantenne di Arco. Vano l'in...,http://www.trentotoday.it/cronaca/incidenti-uo...,2011/11/16,settantenne arco morto pomeriggio schiacciato...,['incidenti'],Arco,"Arco, Comunità Alto Garda e Ledro, Provincia d...","(45.9180319, 10.8860765, 0.0)",45.918032,10.886076,2011,11,16,['incidenti'],incidenti circle y2011
27308,Nasce anche a Trento un gruppo di scout musulmani,Si tratta della quarta esperienza dell'Aism in...,http://www.trentotoday.it/cronaca/societa-grup...,2011/11/16,"zouhaier, tunisino origine, italia 1989, fond...",['società'],Gardolo,"Gardolo, Trento, Territorio Val d'Adige, Provi...","(46.1074737, 11.1100404, 0.0)",46.107474,11.110040,2011,11,16,['società'],società circle y2011


In [17]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols = ["title","subtitle","link","timestamp","text","keywords","address","year","month","day","list_keywords", "classes"]
geojson = df_to_geojson(df,cols )

In [18]:
with open('/home/frabatx/Documents/University/Data_journalism/Project_Article/today_parser/site/src/trento_total.geojson', 'w', encoding='utf-8') as f:
    json.dump(geojson, f, ensure_ascii=True)